In [4]:
# Test SNR Calculation with extracted seizure segments and noise segments from EEG data

import numpy as np
from scipy.signal import welch
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error
import os
import glob
import warnings

def calculate_snr(signal, noise):
    """Calculate Signal-to-Noise Ratio (SNR) in decibels."""
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    if noise_power == 0:
        return float('inf')  # Infinite SNR if no noise
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

In [5]:
# Load EEG data from 
path_seizure_only = r"E:\ML algoritme tl anfaldsdetektion vha HRV\LabView-Results_Excluded_seizures_removed\Responders\Patient 5\recording 1\event01_seizure_only.csv"
path_seizure_ctx = r"E:\ML algoritme tl anfaldsdetektion vha HRV\LabView-Results_Excluded_seizures_removed\Responders\Patient 5\recording 1\event01_seizure_ctx.csv"
path_nonseizure = r"E:\ML algoritme tl anfaldsdetektion vha HRV\LabView-Results_Excluded_seizures_removed\Responders\Patient 5\recording 1\event01_nonseizure.csv"

seizure_only = np.loadtxt(path_seizure_only, delimiter=',', skiprows=1, usecols=1)
seizure_ctx = np.loadtxt(path_seizure_ctx, delimiter=',', skiprows=1, usecols=1)
nonseizure = np.loadtxt(path_nonseizure, delimiter=',', skiprows=1, usecols=1)

# Sanity check: Print shapes of loaded data
print(f"Seizure Only Shape: {seizure_only.shape}")
print(f"Seizure Context Shape: {seizure_ctx.shape}")
print(f"Non-Seizure Shape: {nonseizure.shape}")

Seizure Only Shape: (61440,)
Seizure Context Shape: (184320,)
Non-Seizure Shape: (184320,)


In [6]:
# Calculate SNR for seizure only vs non-seizure
snr_seizure_only = calculate_snr(seizure_only, nonseizure)
print(f"SNR (Seizure Only vs Non-Seizure): {snr_seizure_only:.2f} dB")

SNR (Seizure Only vs Non-Seizure): 9.21 dB


In [7]:
import neurokit2 as nk

# Clean the seizure signal using NeuroKit2
cleaned_seizure = nk.ecg_clean(seizure_only, sampling_rate=512, method="neurokit")
cleaned_seizure_ctx = nk.ecg_clean(seizure_ctx, sampling_rate=512, method="neurokit")
cleaned_nonseizure = nk.ecg_clean(nonseizure, sampling_rate=512, method="neurokit")

noise_seizure = seizure_only - cleaned_seizure
noise_seizure_ctx = seizure_ctx - cleaned_seizure_ctx
noise_nonseizure = nonseizure - cleaned_nonseizure


In [8]:

# Calculate SNR for cleaned seizure vs noise
snr_cleaned_seizure = calculate_snr(seizure_only, noise_seizure)
snr_cleaned_seizure_ctx = calculate_snr(seizure_ctx, noise_seizure_ctx)
snr_cleaned_nonseizure = calculate_snr(nonseizure, noise_nonseizure)

print(f"SNR (Cleaned Seizure vs Noise): {snr_cleaned_seizure:.2f} dB")
print(f"SNR (Cleaned Seizure Context vs Noise): {snr_cleaned_seizure_ctx:.2f} dB")
print(f"SNR (Cleaned Non-Seizure vs Noise): {snr_cleaned_nonseizure:.2f} dB")

SNR (Cleaned Seizure vs Noise): 1.55 dB
SNR (Cleaned Seizure Context vs Noise): 2.19 dB
SNR (Cleaned Non-Seizure vs Noise): 8.52 dB


In [9]:
import numpy as np
from dataclasses import dataclass
from typing import Dict, Optional, Tuple

# Valgfrie afhængigheder
try:
    import scipy.signal as sps
    SCIPY_AVAILABLE = True
except Exception:
    SCIPY_AVAILABLE = False

try:
    import neurokit2 as nk
    NK_AVAILABLE = True
except Exception:
    NK_AVAILABLE = False


@dataclass
class SQIConfig:
    # --- Feasibility ---
    min_duration_s: float = 10.0          # kræv mindst 10 s data
    min_beats: int = 5                    # kræv mindst 5 slag
    min_peak_to_peak_mv: float = 0.1      # min. dynamik efter båndpas (i dine enheder)
    max_saturation_frac: float = 0.01     # andel samples på min/max (≈ flatline)

    # --- Beat-segmenter omkring R-peak ---
    pre_r_s: float = 0.20                 # vindue før R
    post_r_s: float = 0.40                # vindue efter R

    # --- ABC (Average Beat Correlation) ---
    use_median_template: bool = True      # median-template er robust

    # --- RR-konsistens ---
    rr_cv_clip: float = 0.25              # CV over 25% regnes som “dårligt”
    rr_outlier_tol: float = 0.15          # outlier hvis |RR-med| > 15% af median RR

    # --- vægte (w_abc + w_rr_consistency = 1.0) ---
    w_abc: float = 0.50
    w_rr_consistency: float = 0.40

    # --- accept-tærskler ---
    accept_threshold: float = 0.75
    marginal_threshold: float = 0.60


def _butter_bandpass(ecg: np.ndarray, fs: float, low=0.5, high=40.0, order=4) -> np.ndarray:
    if not SCIPY_AVAILABLE:
        return ecg.astype(float)
    nyq = 0.5 * fs
    b, a = sps.butter(order, [low/nyq, high/nyq], btype="band")
    return sps.filtfilt(b, a, ecg.astype(float))


def _detect_rpeaks(ecg: np.ndarray, fs: float) -> np.ndarray:
    """R-peak detektion (NeuroKit2 hvis tilgængelig, ellers simpel fallback)."""
    if NK_AVAILABLE:
        try:
            _, rpeaks = nk.ecg_peaks(ecg, sampling_rate=fs)
            locs = rpeaks.get("ECG_R_Peaks", [])
            return np.asarray(locs, dtype=int)
        except Exception:
            pass
    if SCIPY_AVAILABLE:
        sig = np.abs(ecg - np.median(ecg))
        win = int(0.150 * fs) if fs else 1
        if win > 1:
            sig = sps.medfilt(sig, kernel_size=win | 1)
        height = np.percentile(sig, 85)
        distance = int(0.25 * fs)          # min 240 bpm
        peaks, _ = sps.find_peaks(sig, height=height, distance=distance)
        return peaks.astype(int)
    return np.array([], dtype=int)


def _segment_beats(ecg: np.ndarray, r: np.ndarray, fs: float, pre_s: float, post_s: float):
    pre = int(round(pre_s * fs)); post = int(round(post_s * fs))
    beats = []
    for p in r:
        s, e = p - pre, p + post + 1
        if s >= 0 and e <= len(ecg):
            beats.append(ecg[s:e].astype(float))
    if not beats:
        return np.empty((0, 0)), pre
    return np.vstack(beats), pre


def _zscore_rows(arr: np.ndarray, eps=1e-9) -> np.ndarray:
    mu = arr.mean(axis=1, keepdims=True)
    sd = arr.std(axis=1, keepdims=True); sd[sd < eps] = 1.0
    return (arr - mu) / sd


def _average_beat_correlation(beats: np.ndarray, use_median=True) -> float:
    """Gns. Pearson-korrelation mellem hvert beat og et template (median/mean)."""
    if beats.size == 0 or beats.shape[0] < 2:
        return 0.0
    Bz = _zscore_rows(beats)
    template = np.median(Bz, axis=0) if use_median else np.mean(Bz, axis=0)
    t = template - np.mean(template); t_sd = np.std(t)
    if t_sd == 0:
        return 0.0
    t = t / t_sd
    # Korrelations-koefficient ~ gennemsnitlig punktvis produkt (fordi z-scored)
    r = (Bz * t).sum(axis=1) / (Bz.shape[1] - 1)
    r = np.clip(r, -1.0, 1.0)
    r[r < 0] = 0.0                    # negative “straffes” som 0
    return float(np.mean(r))


def _rr_consistency_score(r: np.ndarray, fs: float, cv_clip: float, out_tol: float):
    """Returnér score∈[0,1], samt CV og outlier-rate for RR."""
    if r.size < 3:
        return 0.0, np.nan, np.nan
    rr = np.diff(r) / fs
    med = np.median(rr)
    if med <= 0:
        return 0.0, np.nan, np.nan
    cv = np.std(rr) / med
    cv_norm = min(cv / cv_clip, 1.0)
    out_rate = np.mean((np.abs(rr - med) > (out_tol * med)).astype(float))
    s_cv = 1.0 - cv_norm
    s_out = 1.0 - out_rate
    return float(np.clip(0.5 * (s_cv + s_out), 0.0, 1.0)), float(cv), float(out_rate)


def ecg_sqi_kuetche23(ecg: np.ndarray, fs: float, cfg: Optional[SQIConfig] = None) -> Dict[str, float]:
    """
    SQI for enkelt-kanals ECG inspireret af Kuetche et al., 2023:
      1) feasibility: varighed, antal slag, dynamik, “flatline”
      2) Average Beat Correlation (ABC): morfologi-konsistens (beat vs. template)
      3) RR-konsistens: CV og outlier-andel
      4) kombineret score i [0,1] og accept/marginal/afvis
    """
    if cfg is None:
        cfg = SQIConfig()
    ecg = np.asarray(ecg, dtype=float)
    N = len(ecg)
    if N == 0 or fs <= 0:
        return {"sqi": 0.0, "accept": False, "marginal": False, "abc": 0.0,
                "rr_consistency": 0.0, "rr_cv": np.nan, "rr_outlier_rate": np.nan,
                "n_beats": 0, "feasible": False}

    # Drop vinduer der er *så* korte at filter/segmentering bliver skrøbeligt
    min_samples_for_filter = int(max(5.0, 3.0 * fs))  # fx ≥ 3 sekunder
    if N < min_samples_for_filter:
        return {"sqi": 0.0, "accept": False, "marginal": False, "abc": 0.0,
                "rr_consistency": 0.0, "rr_cv": np.nan, "rr_outlier_rate": np.nan,
                "n_beats": 0, "feasible": False}

    # 1) feasibility
    ecg_f = _butter_bandpass(ecg, fs, 0.5, 40.0, order=4)
    duration_s = N / fs
    dyn = np.percentile(ecg_f, 99.0) - np.percentile(ecg_f, 1.0)
    x = np.round(ecg_f, 6)
    sat_frac = float(np.mean((x == np.min(x)) | (x == np.max(x))))
    r_locs = _detect_rpeaks(ecg_f, fs)
    n_beats = int(len(r_locs))

    feasible = (
        duration_s >= cfg.min_duration_s and
        n_beats >= cfg.min_beats and
        dyn >= cfg.min_peak_to_peak_mv and
        sat_frac <= cfg.max_saturation_frac
    )
    if not feasible:
        return {"sqi": 0.0, "accept": False, "marginal": False, "abc": 0.0,
                "rr_consistency": 0.0, "rr_cv": np.nan, "rr_outlier_rate": np.nan,
                "n_beats": n_beats, "feasible": False}

    # 2) ABC
    beats, _ = _segment_beats(ecg_f, r_locs, fs, cfg.pre_r_s, cfg.post_r_s)
    if beats.size == 0 or beats.shape[0] < cfg.min_beats:
        return {"sqi": 0.0, "accept": False, "marginal": False, "abc": 0.0,
                "rr_consistency": 0.0, "rr_cv": np.nan, "rr_outlier_rate": np.nan,
                "n_beats": int(beats.shape[0]), "feasible": False}
    abc = float(np.clip(_average_beat_correlation(beats, cfg.use_median_template), 0.0, 1.0))

    # 3) RR-konsistens
    rr_score, rr_cv, rr_out = _rr_consistency_score(r_locs, fs, cfg.rr_cv_clip, cfg.rr_outlier_tol)

    # 4) kombination
    sqi = float(np.clip(cfg.w_abc * abc + cfg.w_rr_consistency * rr_score, 0.0, 1.0))
    accept = bool(sqi >= cfg.accept_threshold)
    marginal = bool((sqi >= cfg.marginal_threshold) and (sqi < cfg.accept_threshold))

    return {
        "sqi": sqi,
        "accept": accept,
        "marginal": marginal,
        "abc": abc,
        "rr_consistency": rr_score,
        "rr_cv": rr_cv,
        "rr_outlier_rate": rr_out,
        "n_beats": n_beats,
        "feasible": True
    }


In [10]:
# from ecg_sqi_kuetche23 import ecg_sqi_kuetche23, SQIConfig

# ecg: 1D numpy-array (rå eller let filtreret); fs: samplingfrekvens (fx 512.0)
result = ecg_sqi_kuetche23(seizure_only, fs=512.0)
print(result)
result = ecg_sqi_kuetche23(seizure_ctx, fs=512.0)
print(result)
result = ecg_sqi_kuetche23(nonseizure, fs=512.0)
print(result)
# -> {'sqi': 0.83, 'accept': True, 'marginal': False, 'abc': 0.88,
#     'rr_consistency': 0.77, 'rr_cv': 0.09, 'rr_outlier_rate': 0.04,
#     'n_beats': 72, 'feasible': True}


{'sqi': 0.8222595399139232, 'accept': True, 'marginal': False, 'abc': 0.9886032306071534, 'rr_consistency': 0.8198948115258662, 'rr_cv': 0.07415664047984144, 'rr_outlier_rate': 0.06358381502890173, 'n_beats': 174, 'feasible': True}
{'sqi': 0.6148945417962013, 'accept': False, 'marginal': True, 'abc': 0.9831775715114437, 'rr_consistency': 0.30826439010119866, 'rr_cv': 0.2266548419864377, 'rr_outlier_rate': 0.47685185185185186, 'n_beats': 433, 'feasible': True}
{'sqi': 0.8437988582519216, 'accept': True, 'marginal': False, 'abc': 0.9977438675913168, 'rr_consistency': 0.8623173111406577, 'rr_cv': 0.061422946803558406, 'rr_outlier_rate': 0.02967359050445104, 'n_beats': 338, 'feasible': True}


In [11]:
from src.hrv_epatch.io.tdms_pipeline import load_tdms

path_full_ecg_patient5 = r"E:\ML algoritme tl anfaldsdetektion vha HRV\ePatch - Corrected\Patients ePatch data\Patient 5\recording 1\Patient 5_1.tdms"

pd_data, meta_data = load_tdms(path_full_ecg_patient5)
print(pd_data[:5])  # Print the first 5 elements of the NumPy array
print(meta_data.start_time)

[ -70.27081712   -1.37785916   37.2021973   -45.46935225 -144.67521172]
2016-10-12 11:05:02


In [12]:
fs = meta_data.fs

In [13]:
pd_data_short_copy = pd_data[:500000]  # Shorten for testing purposes

In [14]:
ecg_sqi_kuetche23(pd_data_short_copy, fs)

{'sqi': 0.18759358725284814,
 'accept': False,
 'marginal': False,
 'abc': 0.32483752415604594,
 'rr_consistency': 0.06293706293706292,
 'rr_cv': 27.481785079590153,
 'rr_outlier_rate': 0.8741258741258742,
 'n_beats': 144,
 'feasible': True}

In [15]:
ecg_sqi_kuetche23(pd_data, fs)

{'sqi': 0.6313168739483627,
 'accept': False,
 'marginal': True,
 'abc': 0.9875049591103351,
 'rr_consistency': 0.34391098598298786,
 'rr_cv': 0.997505587733245,
 'rr_outlier_rate': 0.3121780280340242,
 'n_beats': 166941,
 'feasible': True}

In [16]:
import numpy as np
import math
from typing import List, Dict

# --- udskift i _detect_rpeaks ---
def _detect_rpeaks(ecg: np.ndarray, fs: float) -> np.ndarray:
    import numpy as _np
    # NeuroKit2 først
    if NK_AVAILABLE:
        try:
            _, rpeaks = nk.ecg_peaks(ecg, sampling_rate=fs)
            locs = rpeaks.get("ECG_R_Peaks", [])
            return _np.asarray(locs, dtype=int)
        except Exception:
            pass
    # Robust fallback: clamp kernel <= vindueslængde og tving ulig (odd)
    if SCIPY_AVAILABLE:
        sig = _np.abs(ecg - _np.median(ecg))
        max_kernel = max(3, (len(sig) // 5) | 1)     # aldrig større end vinduet/5
        win = int(max(3, min(max_kernel, round(0.150 * fs)) )) | 1
        try:
            sig = sps.medfilt(sig, kernel_size=win)
        except Exception:
            pass  # hvis for kort, kør uden medfilt
        height = _np.percentile(sig, 85) if sig.size else 0.0
        distance = max(1, int(0.25 * fs))
        peaks, _ = sps.find_peaks(sig, height=height, distance=distance)
        return peaks.astype(int)
    return _np.array([], dtype=int)

# --- udskift i _segment_beats ---
def _segment_beats(ecg: np.ndarray, r_locs: np.ndarray, fs: float, pre_s: float, post_s: float):
    pre = int(round(pre_s * fs)); post = int(round(post_s * fs))
    win = pre + post + 1
    if win < 3:  # for smalle beats giver tomme/ustabile korrelationer
        return np.empty((0, 0)), pre
    beats = []
    for r in r_locs:
        s, e = r - pre, r + post + 1
        if s >= 0 and e <= len(ecg):
            seg = ecg[s:e].astype(float)
            if np.isfinite(seg).all():    # drop beats der kun er NaN/Inf
                beats.append(seg)
    if not beats:
        return np.empty((0, 0)), pre
    return np.vstack(beats), pre

# --- udskift i _zscore_rows ---
def _zscore_rows(arr: np.ndarray, eps: float = 1e-9) -> np.ndarray:
    if arr.size == 0 or arr.shape[1] == 0:
        return arr
    mu = arr.mean(axis=1, keepdims=True)
    sd = arr.std(axis=1, keepdims=True)
    sd[~np.isfinite(sd)] = 1.0
    sd[sd < eps] = 1.0
    Z = (arr - mu) / sd
    Z[~np.isfinite(Z)] = 0.0
    return Z

# --- udskift i _average_beat_correlation ---
def _average_beat_correlation(beats: np.ndarray, use_median: bool = True) -> float:
    # Mindst 2 beats og mindst 3 samples per beat
    if beats.size == 0 or beats.shape[0] < 2 or beats.shape[1] < 3:
        return 0.0
    beats_z = _zscore_rows(beats)
    if beats_z.size == 0 or beats_z.shape[1] < 3:
        return 0.0
    template = np.median(beats_z, axis=0) if use_median else np.mean(beats_z, axis=0)
    t = template - np.mean(template)
    t_sd = np.std(t)
    if not np.isfinite(t_sd) or t_sd == 0.0:
        return 0.0
    t = t / t_sd
    r = (beats_z * t).sum(axis=1) / max(1, (beats_z.shape[1] - 1))
    r = np.clip(r, -1.0, 1.0)
    r[~np.isfinite(r)] = 0.0
    r[r < 0] = 0.0
    return float(np.mean(r)) if r.size else 0.0


In [17]:
def run_sqi_over_windows(ecg, fs, win_s=10.0, step_s=5.0, min_beats=5, verbose=False):
    ecg = np.asarray(ecg, dtype=float)
    if ecg.size == 0 or fs <= 0:
        return []
    # Erstat NaN/Inf inden sliding
    if not np.isfinite(ecg).all():
        good = np.isfinite(ecg)
        fill = np.nanmedian(ecg[good]) if good.any() else 0.0
        ecg = np.where(np.isfinite(ecg), ecg, fill)

    win = int(round(win_s * fs)); step = int(round(step_s * fs))
    if win < int(3 * fs) or win <= 0 or step <= 0 or win > len(ecg):
        if verbose: print("[SQI] window too short/long for this signal.")
        return []

    cfg = SQIConfig(min_beats=min_beats)
    rows = []
    for s in range(0, len(ecg) - win + 1, step):
        seg = ecg[s:s+win]
        # Hurtig sanity: hvis <50% af samples har variation -> skip
        if np.nanstd(seg) < 1e-9:
            rows.append({"t0": s/fs, "t1": (s+win)/fs, "sqi": np.nan,
                         "accept": False, "marginal": False,
                         "n_beats": 0, "feasible": False, "reason": "flat"})
            continue
        try:
            res = ecg_sqi_kuetche23(seg, fs, cfg=cfg)
        except Exception as ex:
            rows.append({"t0": s/fs, "t1": (s+win)/fs, "sqi": np.nan,
                         "accept": False, "marginal": False,
                         "n_beats": 0, "feasible": False, "reason": f"exception:{ex}"})
            continue

        reason = ""
        if not res["feasible"]:
            reason = "too_few_beats" if res.get("n_beats", 0) < cfg.min_beats else "feasibility_failed"

        rows.append({"t0": s/fs, "t1": (s+win)/fs, "sqi": float(res["sqi"]) if np.isfinite(res["sqi"]) else np.nan,
                     "accept": bool(res["accept"]), "marginal": bool(res["marginal"]),
                     "n_beats": int(res.get("n_beats", 0)), "feasible": bool(res["feasible"]),
                     "reason": reason})
    return rows


In [18]:
series = run_sqi_over_windows(pd_data, fs=512.0, win_s=30.0, step_s=10.0, min_beats=5, verbose=True)
# Til DataFrame:
import pandas as pd
df_sqi = pd.DataFrame(series)
display(df_sqi.head())

# Fx filtrer acceptable vinduer:
df_ok = df_sqi[(df_sqi["accept"])]


c:\Users\kvjkv\anaconda3\envs\mast\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\kvjkv\anaconda3\envs\mast\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\kvjkv\anaconda3\envs\mast\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\kvjkv\anaconda3\envs\mast\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\kvjkv\anaconda3\envs\mast\Lib\site-packages\numpy\_core\fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\kvjkv\anaconda3\envs\mast\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountere

,t0,t1,sqi,accept,marginal,n_beats,feasible,reason
0,0.0,30.0,0.190714,False,False,33,True,
1,10.0,40.0,0.225215,False,False,31,True,
2,20.0,50.0,0.188661,False,False,28,True,
3,30.0,60.0,0.171285,False,False,18,True,
4,40.0,70.0,0.170711,False,False,16,True,


In [19]:
df_ok

,t0,t1,sqi,accept,marginal,n_beats,feasible,reason
127,1270.0,1300.0,0.787210,True,False,32,True,
131,1310.0,1340.0,0.851214,True,False,34,True,
132,1320.0,1350.0,0.853197,True,False,34,True,
133,1330.0,1360.0,0.834321,True,False,32,True,
135,1350.0,1380.0,0.877046,True,False,30,True,
...,...,...,...,...,...,...,...,...
15919,159190.0,159220.0,0.867744,True,False,37,True,
15920,159200.0,159230.0,0.860539,True,False,38,True,
15921,159210.0,159240.0,0.869299,True,False,38,True,
15922,159220.0,159250.0,0.850126,True,False,38,True,


In [20]:
# Show statistics
df_ok.describe()

,t0,t1,sqi,n_beats
count,15329.000000,15329.000000,15329.000000,15329.000000
mean,80952.492661,80982.492661,0.857977,31.202427
std,45547.579173,45547.579173,0.026579,4.791700
min,1270.000000,1300.000000,0.750019,22.000000
25%,41570.000000,41600.000000,0.850604,28.000000
50%,81520.000000,81550.000000,0.866558,30.000000
75%,120450.000000,120480.000000,0.875768,34.000000
max,159230.000000,159260.000000,0.893842,75.000000


In [21]:
# Calculate what fraction of windows are acceptable
fraction_acceptable = len(df_ok) / len(df_sqi) if len(df_sqi) > 0 else 0.0
print(f"Fraction of acceptable windows: {fraction_acceptable:.2%}")

Fraction of acceptable windows: 95.19%


In [22]:
# Show non-acceptable windows
df_not_ok = df_sqi[~df_sqi["accept"]]
# Find consecutive "boxes" of bad signal
bad_signal_indices = df_not_ok.index.to_series()
consecutive_groups = (bad_signal_indices != bad_signal_indices.shift() + 1).cumsum()

# Group by consecutive indices and print start and end time of each group
for _, group in bad_signal_indices.groupby(consecutive_groups):
    start_time = df_not_ok.loc[group.iloc[0], "t0"]
    end_time = df_not_ok.loc[group.iloc[-1], "t1"]
    print(f"Start time: {start_time:.2f}s, End time: {end_time:.2f}s")

Start time: 0.00s, End time: 1290.00s
Start time: 1280.00s, End time: 1330.00s
Start time: 1340.00s, End time: 1370.00s
Start time: 1700.00s, End time: 1750.00s
Start time: 1910.00s, End time: 1940.00s
Start time: 1970.00s, End time: 2000.00s
Start time: 2000.00s, End time: 2040.00s
Start time: 2840.00s, End time: 2890.00s
Start time: 2970.00s, End time: 3010.00s
Start time: 3130.00s, End time: 3170.00s
Start time: 3690.00s, End time: 3760.00s
Start time: 4350.00s, End time: 4380.00s
Start time: 4410.00s, End time: 4450.00s
Start time: 5180.00s, End time: 5220.00s
Start time: 5370.00s, End time: 5410.00s
Start time: 5690.00s, End time: 5720.00s
Start time: 6820.00s, End time: 6860.00s
Start time: 7040.00s, End time: 7080.00s
Start time: 7780.00s, End time: 7810.00s
Start time: 8100.00s, End time: 8150.00s
Start time: 8400.00s, End time: 8470.00s
Start time: 8510.00s, End time: 8540.00s
Start time: 8690.00s, End time: 8740.00s
Start time: 8870.00s, End time: 8910.00s
Start time: 10320.0

In [23]:
# Function to retrieve information for all bins within a specific window length from a given time
def get_sqi_info_for_window(df, start_time_in_s, window_length_in_s):
    end_time_in_s = start_time_in_s + window_length_in_s
    rows = df[(df['t0'] >= start_time_in_s) & (df['t1'] <= end_time_in_s)]
    if not rows.empty:
        return rows
    else:
        return f"No data found for the time range {start_time_in_s}s to {end_time_in_s}s"

# Example usage
start_time_in_s = 73315.0  # Replace with your desired start time in seconds
window_length_in_s = 120.0  # Length of the window in seconds

info = get_sqi_info_for_window(df_sqi, start_time_in_s, window_length_in_s)
print(info)

           t0       t1       sqi  accept  marginal  n_beats  feasible reason
7332  73320.0  73350.0  0.816625    True     False       44      True       
7333  73330.0  73360.0  0.854196    True     False       42      True       
7334  73340.0  73370.0  0.861595    True     False       42      True       
7335  73350.0  73380.0  0.851253    True     False       43      True       
7336  73360.0  73390.0  0.860912    True     False       44      True       
7337  73370.0  73400.0  0.866442    True     False       44      True       
7338  73380.0  73410.0  0.814799    True     False       42      True       
7339  73390.0  73420.0  0.784850    True     False       42      True       
7340  73400.0  73430.0  0.774906    True     False       43      True       


In [24]:
# from src.hrv_epatch.io.tdms import extract_tdms_channel

# extract_tdms_channel(path_full_ecg_patient5)

In [25]:
from datetime import datetime as dt

# Calculate sec between two datetimes
def calculate_time_difference(start_time, end_time):
    time_diff = end_time - start_time
    return time_diff.total_seconds()

start_time = dt(2016, 10, 12, 11, 5, 2)
print(start_time)
end_time = dt(2016, 10, 13, 7, 26, 57)
print(end_time)

seconds_diff = calculate_time_difference(start_time, end_time)
print(f"Time difference in seconds: {seconds_diff} seconds")

2016-10-12 11:05:02
2016-10-13 07:26:57
Time difference in seconds: 73315.0 seconds
